# Public functions 

In [1]:
import pandas as pd 
import numpy as np
from functools import lru_cache
import os
from tqdm import tqdm 

In [2]:
sasrec_session_candidates_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/SASRec_Next/kdd_cup_2023/2023-04-01-14-13-51.parquet'
co_graph_item_candidates_path = '/root/autodl-tmp/huangxu/Amazon-KDDCUP-23/co-occurrence_graph/item_candidates.ftr'
retromae_session_candidates_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/retromae_UK_valid_prediction.parquet'
roberta_session_candidates_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/roberta/roberta_valid_300_with_score.parquet'
valid_inter_feat_path = '../data_for_recstudio/all_task_1_valid_inter_feat.csv'
valid_sessions_path = '../data_for_recstudio/all_task_1_valid_sessions.csv'
product_data_path = '../raw_data/products_train.csv'

In [3]:
@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_sasrec_session_candidates():
    return pd.read_parquet(sasrec_session_candidates_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_retromae_session_candidates():
    return pd.read_parquet(retromae_session_candidates_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_roberta_session_candidates():
    return pd.read_parquet(roberta_session_candidates_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_co_graph_item_candidates():
    return pd.read_feather(co_graph_item_candidates_path)

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(product_data_path)

In [11]:
def get_session_last_item(session_df):
    last_items = []
    num_sessions = len(session_df)
    for i in tqdm(range(num_sessions)):
        sess = session_df.iloc[i]
        sess_prev_items = sess['prev_items']
        
        product_list = sess_prev_items.strip('[]').split(' ')
        last_item = product_list[-1].strip("'\n")

        last_items.append(last_item)
    return last_items 


# Statistic between SASRec and co-graph 

In [4]:
sasrec_session_candidates_df = read_sasrec_session_candidates()
sasrec_session_candidates_df.head(10), len(sasrec_session_candidates_df)

(  locale                                         candidates sess_id
 0     UK  [B06XG1LZ6Z, B06XGD9VLV, B076PN1SKG, B01MYUDYP...       0
 1     JP  [B09LCPT9DQ, B09MRYK5CV, B0BB5VQ1L8, B084T9C6W...       1
 2     UK  [B09XBS6WCX, B01EYGW86Y, B01MDOBUCC, B01C5YK17...       2
 3     UK  [0241572614, 1406392979, 024157563X, 024147681...       3
 4     JP  [B0B6PF619D, B0B6P77ZRN, B0B6P2PCMP, B0B6NY4PN...       4
 5     DE  [B0BD48G63Q, B0BD3DGNT9, B0B685KHK6, B0B54PQKG...       5
 6     DE  [B07Q82LRDK, B082XSF8XF, B00H37AVY8, B01EGP3LV...       6
 7     UK  [B013SL2712, B07V5LWSN5, B00CL6353A, B07W6JP97...       7
 8     JP  [B0BFPGHSYX, B09ZTN81QH, B00B57A5IY, B00LE7TO0...       8
 9     DE  [B07MH3K3S8, B07KQHHYQC, B00Z0BTBEA, B07YLZ67Q...       9,
 361581)

In [13]:
co_graph_item_candidates_df = read_co_graph_item_candidates()
co_graph_item_candidates_df.head(10), len(co_graph_item_candidates_df)

(           id                                         candidates
 0  B005ZSSN10  [B07Q1YXVWH, B0B94HRCXK, B09VXRZKWB, B09LCVHCV...
 1  B08PRYN6LD  [B08PSBK59Y, B079RJTR2G, B01N7RTYXE, B08HHD25M...
 2  B09MBZJ48V  [B07VR16HF9, B07FL7GVZG, B09DPR2LZW, B0B9S2FDD...
 3  B08ZN6F26S  [B08PVG787Z, B07PJVGVSM, B08ZN6F26S, 140834646...
 4  B094DGRV7D  [B08KTP6517, B094DGRV7D, B092DFVDC4, B09NXJ9KS...
 5  B09JNNBDH5  [B08K4214KB, B01MG62Z5M, B0963J155G, B008F8PMU...
 6  B08R62WZ1Y  [B09NVJGT2W, B09TB6TK6D, B08NY2NM1M, B08R62WZ1...
 7  B09WK4YNX8  [B0B2X1YX72, B094R3R9XH, B0BDML9477, B0BCPD9YY...
 8  B07KTKFYYS  [B08V1KXBQD, B0B94HRCXK, B07RHT52HX, B0B7J4WSM...
 9  B0B9MPKYJK  [B09HWSDTM3, B09C6H17PK, B08LR3G17D, B09SWTJZH...,
 1410675)

In [16]:
valid_sessions_df = read_valid_sessions()
valid_sessions_df.head(10)

,prev_items,next_item,locale
0,['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VS...,B06XG1LZ6Z,UK
1,['B00390YWXE' 'B00390YWXE' 'B09WM9W6WQ'],B01MSUI4FE,JP
2,['B01BM9V6H8' 'B01MG55XDR' 'B07VYSSRL7'],B01M6625ME,UK
3,['B092ZG24S7' 'B09BNHWWZM' 'B08CB1WG5M' '17880...,0241558573,UK
4,['B0B6NY5RM8' 'B09BJGBBBR'],B09BJF6N8K,JP
5,['B0B84YXQLV' 'B09G19MPQH' 'B07GRTY2ZZ' 'B08CB...,B0BD48G63Q,DE
6,['B001O51JCS' 'B001O51JCS'],B0045DNZ9Q,DE
7,['B07V5LWSN5' 'B0BF1CGGSJ' 'B07W4DH45P' 'B08P3...,B07W6J5FB2,UK
8,['B019WI5XXO' 'B08KRZX6DG'],B087BDR5GP,JP
9,['B0874RWZVB' 'B07MP1ZSLJ'],B07Q65PXXP,DE


In [18]:
co_graph_item_candidates_df = co_graph_item_candidates_df.set_index('id', drop=False)
co_graph_item_candidates_df.head(5)

,id,candidates
id,,
B005ZSSN10,B005ZSSN10,"[B07Q1YXVWH, B0B94HRCXK, B09VXRZKWB, B09LCVHCV..."
B08PRYN6LD,B08PRYN6LD,"[B08PSBK59Y, B079RJTR2G, B01N7RTYXE, B08HHD25M..."
B09MBZJ48V,B09MBZJ48V,"[B07VR16HF9, B07FL7GVZG, B09DPR2LZW, B0B9S2FDD..."
B08ZN6F26S,B08ZN6F26S,"[B08PVG787Z, B07PJVGVSM, B08ZN6F26S, 140834646..."
B094DGRV7D,B094DGRV7D,"[B08KTP6517, B094DGRV7D, B092DFVDC4, B09NXJ9KS..."


generate session candidates from co-graph item condidates 

In [19]:
valid_sessions_df['prev_items'][0]

"['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VSN9GLS' 'B0BJ6V797Y'\n 'B09VSG9DCG' 'B077XGDMD2']"

In [21]:
from tqdm import tqdm 

def get_session_last_item(session_df):
    last_items = []
    num_sessions = len(session_df)
    for i in tqdm(range(num_sessions)):
        sess = session_df.iloc[i]
        sess_prev_items = sess['prev_items']
        
        product_list = sess_prev_items.strip('[]').split(' ')
        last_item = product_list[-1].strip("'\n")

        last_items.append(last_item)
    return last_items 

last_items = get_session_last_item(valid_sessions_df)
last_items[:10]

100%|██████████| 361581/361581 [00:09<00:00, 36750.18it/s]


['B077XGDMD2',
 'B09WM9W6WQ',
 'B07VYSSRL7',
 '1788009975',
 'B09BJGBBBR',
 'B0BD489B72',
 'B001O51JCS',
 'B013SL1YXO',
 'B08KRZX6DG',
 'B07MP1ZSLJ']

In [31]:
co_graph_session_candidates_df = co_graph_item_candidates_df.loc[last_items]
co_graph_session_candidates_df[:10]

,id,candidates
id,,
B077XGDMD2,B077XGDMD2,"[B077XGDMD2, B06XGDZVZR, B06XG1LZ6Z, B076PN1SK..."
B09WM9W6WQ,B09WM9W6WQ,"[B09LCPT9DQ, B09WM9W6WQ, B092D5HM5S, B09MRYK5C..."
B07VYSSRL7,B07VYSSRL7,"[B09XBS6WCX, B01M6625ME, B094W9KT4Y, B09SHSZ2C..."
1788009975,1788009975,"[1839941960, 1788009975, 0241563437, 000853222..."
B09BJGBBBR,B09BJGBBBR,"[B0B6PF619D, B0B6NRKVZF, B0B6NY4PNC, B0B6P77ZR..."
B0BD489B72,B0BD489B72,"[B0BD48G63Q, B0B6ZNHCVV, B0B685KHK6, B0B7RNG4X..."
B001O51JCS,B001O51JCS,"[B07Q82LRDK, B08BR288Z7, B09MP8BCYP, B01NCX3W3..."
B013SL1YXO,B013SL1YXO,"[B013SL1YXO, B08FBZ8QSL, B085WYXXLF, B087MV18C..."
B08KRZX6DG,B08KRZX6DG,"[B00LE7TO0K, B00B57A5IY, B08KRZX6DG, B086LGXKY..."


In [40]:
num_sessions = len(valid_sessions_df)
co_currence_rates = []
for i in tqdm(range(num_sessions)):
    co_graph_set = set(co_graph_session_candidates_df.iloc[i]['candidates'])
    sasrec_set = set(sasrec_session_candidates_df.iloc[i]['candidates'])
    co_currence_set = co_graph_set.intersection(sasrec_set)
    co_currence_rates.append(len(co_currence_set) * 1.0 / 300.0)
co_currence_rate = np.array(co_currence_rates).mean()

100%|██████████| 361581/361581 [02:33<00:00, 2348.93it/s]


In [49]:
def co_currence(length):
    num_sessions = len(valid_sessions_df)
    co_currence_rates = []
    for i in tqdm(range(num_sessions)):
        co_graph_set = set(co_graph_session_candidates_df.iloc[i]['candidates'][:length])
        sasrec_set = set(sasrec_session_candidates_df.iloc[i]['candidates'][:length])
        co_currence_set = co_graph_set.intersection(sasrec_set)
        co_currence_rates.append(len(co_currence_set) * 1.0 / length * 1.0)
    return np.array(co_currence_rates).mean()

In [50]:
co_currence_rate_300 = co_currence(300)
co_currence_rate_100 = co_currence(100)
co_currence_rate_10 = co_currence(10)
co_currence_rate_300, co_currence_rate_100, co_currence_rate_10

100%|██████████| 361581/361581 [01:04<00:00, 5609.18it/s]


(0.06729248310429292, 0.1313047975419062, 0.2583213719747442)

In [56]:
def co_currence_2(length):
    num_sessions = len(valid_sessions_df)
    co_currence_rates = []
    for i in tqdm(range(num_sessions)):
        co_graph_set = set(co_graph_session_candidates_df.iloc[i]['candidates'][:length])
        sasrec_set = set(sasrec_session_candidates_df.iloc[i]['candidates'][:length])
        co_currence_set = co_graph_set.intersection(sasrec_set)
        co_currence_rates.append(len(co_currence_set) * 1.0 / 300)
    return np.array(co_currence_rates).mean()

In [57]:
co_currence_rate_300_2 = co_currence_2(300)
co_currence_rate_100_2 = co_currence_2(100)
co_currence_rate_10_2 = co_currence_2(10)
co_currence_rate_300_2, co_currence_rate_100_2, co_currence_rate_10_2

100%|██████████| 361581/361581 [01:03<00:00, 5662.77it/s]


(0.06729248310429292, 0.043768265847302076, 0.00861071239915814)

In [34]:
def cal_hit_and_mrr(ground_truth_list, candidates_list):
    hits, mrrs = [], []
    for i in tqdm(range(len(ground_truth_list))):
        ground_truth = ground_truth_list.iloc[i]
        candidates = candidates_list.iloc[i]
        hit, mrr = 0.0, 0.0
        for j in range(len(candidates)):
            if ground_truth == candidates[j]:
                hit = 1.0
                mrr = 1.0 / (j + 1)
                break
        hits.append(hit)
        mrrs.append(mrr)
    return np.array(hits).mean(), np.array(mrrs).mean()

In [36]:
# co-graph hit ratio and mrr
co_graph_hit_ratio, co_graph_mrr = cal_hit_and_mrr(valid_sessions_df['next_item'], co_graph_session_candidates_df['candidates'])
co_graph_hit_ratio, co_graph_mrr

100%|██████████| 361581/361581 [00:08<00:00, 41247.47it/s]


(0.5527115639372644, 0.27416820624916205)

In [5]:
# sasrec hit ratio and mrr
sasrec_hit_ratio, sasrec_mrr = cal_hit_and_mrr(valid_sessions_df['next_item'], sasrec_session_candidates_df['candidates'])
sasrec_hit_ratio, sasrec_mrr

NameError: name 'cal_hit_and_mrr' is not defined

In [44]:
# hit concurrently
def cal_co_hit(ground_truth_list, candidates_list_1, candidates_list_2):
    hits = []
    for i in tqdm(range(len(ground_truth_list))):
        ground_truth = ground_truth_list.iloc[i]
        candidates_1 = candidates_list_1.iloc[i]
        candidates_2 = candidates_list_2.iloc[i]
        hit = 0.0
        if (ground_truth in candidates_1) and (ground_truth in candidates_2):
            hit = 1.0
        hits.append(hit)
    return np.array(hits).mean()

In [45]:
co_hit = cal_co_hit(valid_sessions_df['next_item'], co_graph_session_candidates_df['candidates'], sasrec_session_candidates_df['candidates'])
co_hit

100%|██████████| 361581/361581 [00:35<00:00, 10115.28it/s]


0.43221297579242274

In [43]:
# hit or
def cal_or_hit(ground_truth_list, candidates_list_1, candidates_list_2):
    hits = []
    for i in tqdm(range(len(ground_truth_list))):
        ground_truth = ground_truth_list.iloc[i]
        candidates_1 = candidates_list_1.iloc[i]
        candidates_2 = candidates_list_2.iloc[i]
        hit = 0.0
        if (ground_truth in candidates_1) or (ground_truth in candidates_2):
            hit = 1.0
        hits.append(hit)
    return np.array(hits).mean()

In [53]:
or_hit = cal_or_hit(valid_sessions_df['next_item'], co_graph_session_candidates_df['candidates'], sasrec_session_candidates_df['candidates'])
or_hit

100%|██████████| 361581/361581 [00:34<00:00, 10373.10it/s]


0.7421601245640672

In [74]:
co_graph_session_first = co_graph_session_candidates_df['candidates'].apply(lambda x : x[:2])
co_graph_session_first

id
B077XGDMD2    [B077XGDMD2, B06XGDZVZR]
B09WM9W6WQ    [B09LCPT9DQ, B09WM9W6WQ]
B07VYSSRL7    [B09XBS6WCX, B01M6625ME]
1788009975    [1839941960, 1788009975]
B09BJGBBBR    [B0B6PF619D, B0B6NRKVZF]
                        ...           
B08L8N8HDR    [B01A955L8G, B08NYPTF96]
B09D76FT9D    [B09BBX1T4S, B09D76FT9D]
B0BC2K5CY1    [B07KLCY8NF, B0BC38GHB4]
B07Q2CNLY3    [B08RQDVX71, B08RQR2NPB]
B08427PFR5    [B00P0VKZ7W, B016UZOCX4]
Name: candidates, Length: 361581, dtype: object

In [75]:
co_graph_first_hit, co_graph_first_mrr = cal_hit_and_mrr(valid_sessions_df['next_item'], co_graph_session_first)
co_graph_first_hit, co_graph_first_mrr

100%|██████████| 361581/361581 [00:18<00:00, 19742.36it/s]


(0.2640321255818198, 0.23255508447623077)

In [76]:
sasrec_session_first = sasrec_session_candidates_df['candidates'].apply(lambda x : x[:2])
sasrec_session_first

0         [B06XG1LZ6Z, B06XGD9VLV]
1         [B09LCPT9DQ, B09MRYK5CV]
2         [B09XBS6WCX, B01EYGW86Y]
3         [0241572614, 1406392979]
4         [B0B6PF619D, B0B6P77ZRN]
                    ...           
361576    [B0050IG9DE, B0B7LVKNK8]
361577    [B09B9V4PXC, B09BCM5NL1]
361578    [B0BC38GHB4, B07KLCY8NF]
361579    [B08RQR2NPB, B08RQDVX71]
361580    [B0095FMJE6, B016UZOCX4]
Name: candidates, Length: 361581, dtype: object

In [77]:
sasrec_first_hit, sasrec_first_mrr = cal_hit_and_mrr(valid_sessions_df['next_item'], sasrec_session_first)
sasrec_first_hit, sasrec_first_mrr

100%|██████████| 361581/361581 [00:15<00:00, 23871.29it/s]


(0.2694389362272907, 0.23750280020244427)

In [78]:
cal_or_hit(valid_sessions_df['next_item'], sasrec_session_first, co_graph_session_first)

100%|██████████| 361581/361581 [00:25<00:00, 14462.34it/s]


0.36078776263133294

In [87]:
import copy 
merge_session_candidates_df = copy.deepcopy(sasrec_session_candidates_df)
for i in tqdm(range(len(merge_session_candidates_df))):
    sasrec_candidates = merge_session_candidates_df.iloc[i]['candidates']
    co_graph_first = co_graph_session_candidates_df.iloc[i]['candidates'][0:1]
    merge_candidates = np.concatenate([sasrec_candidates[:1], co_graph_first, sasrec_candidates[1:-1]])
    merge_session_candidates_df.iloc[i]['candidates'] = merge_candidates

100%|██████████| 361581/361581 [01:48<00:00, 3337.89it/s]


In [88]:
sasrec_session_candidates_df.head(5)

,locale,candidates,sess_id
0,UK,"[B06XG1LZ6Z, B06XGD9VLV, B076PN1SKG, B01MYUDYP...",0
1,JP,"[B09LCPT9DQ, B09MRYK5CV, B0BB5VQ1L8, B084T9C6W...",1
2,UK,"[B09XBS6WCX, B01EYGW86Y, B01MDOBUCC, B01C5YK17...",2
3,UK,"[0241572614, 1406392979, 024157563X, 024147681...",3
4,JP,"[B0B6PF619D, B0B6P77ZRN, B0B6P2PCMP, B0B6NY4PN...",4


In [89]:
merge_session_candidates_df.head(5)

,locale,candidates,sess_id
0,UK,"[B06XG1LZ6Z, B077XGDMD2, B06XGD9VLV, B076PN1SK...",0
1,JP,"[B09LCPT9DQ, B09LCPT9DQ, B09MRYK5CV, B0BB5VQ1L...",1
2,UK,"[B09XBS6WCX, B09XBS6WCX, B01EYGW86Y, B01MDOBUC...",2
3,UK,"[0241572614, 1839941960, 1406392979, 024157563...",3
4,JP,"[B0B6PF619D, B0B6PF619D, B0B6P77ZRN, B0B6P2PCM...",4


In [91]:
cal_hit_and_mrr(valid_sessions_df['next_item'], merge_session_candidates_df['candidates'])

100%|██████████| 361581/361581 [00:26<00:00, 13856.09it/s]


(0.6616083256587044, 0.2897622701720241)

In [95]:
import copy 
merge_session_candidates_df_2 = copy.deepcopy(sasrec_session_candidates_df)
for i in tqdm(range(len(merge_session_candidates_df_2))):
    sasrec_candidates = merge_session_candidates_df_2.iloc[i]['candidates']
    co_graph_pre = co_graph_session_candidates_df.iloc[i]['candidates'][0:2]
    merge_candidates = np.concatenate([sasrec_candidates[:2], co_graph_pre, sasrec_candidates[2:-1]])
    merge_session_candidates_df_2.iloc[i]['candidates'] = merge_candidates

100%|██████████| 361581/361581 [01:45<00:00, 3431.97it/s]


In [96]:
cal_hit_and_mrr(valid_sessions_df['next_item'], merge_session_candidates_df_2['candidates'])

100%|██████████| 361581/361581 [00:25<00:00, 14448.97it/s]


(0.672994432782696, 0.2886784557162462)

In [94]:
merge_session_candidates_df_2.head(5)

,locale,candidates,sess_id
0,UK,"[B06XG1LZ6Z, B06XGD9VLV, B06XG1LZ6Z, B06XGD9VL...",0
1,JP,"[B09LCPT9DQ, B09MRYK5CV, B09LCPT9DQ, B09MRYK5C...",1
2,UK,"[B09XBS6WCX, B01EYGW86Y, B09XBS6WCX, B01EYGW86...",2
3,UK,"[0241572614, 1406392979, 0241572614, 140639297...",3
4,JP,"[B0B6PF619D, B0B6P77ZRN, B0B6PF619D, B0B6P77ZR...",4


## merge co graph prediction and sasrec prediction

In [100]:
sasrec_prediction_path = '../predictions/SASRec_Next/kdd_cup_2023/2023-04-01-06-24-57.parquet'
co_graph_prediction_path = '../predictions/co_graph/co_occurence_predict_task1.parquet'

In [101]:
@lru_cache(maxsize=1)
def read_sasrec_prediction():
    return pd.read_parquet(sasrec_prediction_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_co_graph_prediction():
    return pd.read_parquet(co_graph_prediction_path, engine='pyarrow')

In [102]:
sasrec_prediction = read_sasrec_prediction()
co_graph_prediction = read_co_graph_prediction()
print(sasrec_prediction.head(5))
print(co_graph_prediction.head(5))

  locale                               next_item_prediction
0     DE  [B07LG5T3V9, B099NS1XPG, B099NR3X6D, B08QYYBTM...
1     DE  [B004ZXMV4Q, B095TQTZXY, B08BZCKDKQ, B010MJNUZ...
2     DE  [B0B5QNFWJ1, B0BJF4KGCN, B00C3TEAPM, B099277D7...
3     DE  [3772476953, 395535086X, B0829LZFT1, B089K81TD...
4     DE  [B09J8V9RQQ, B09J8VPTTW, B09J8SKX9G, B09YGXJJZ...
  locale                               next_item_prediction
0     DE  [B099NS1XPG, B099NR3X6D, B07LG5T3V9, B08WYRNVT...
1     DE  [B004ZXMV4Q, B08H93ZRK9, B0BFJGXWDV, B0B1MPZWJ...
2     DE  [B0BBVB89CS, B0B5QNFWJ1, B099277D7Q, B08HM16Y5...
3     DE  [3772477917, B09KF2JR9F, B09MYL7SHM, B0829LZFT...
4     DE  [B09J8SKX9G, B0B5XG7SWR, B09J8VPTTW, B09YGXJJZ...


In [106]:
def merge_predictions(prediction_1, prediction_2):
    import copy 
    merged_prediction = copy.deepcopy(prediction_1)
    for i in tqdm(range(len(merged_prediction))):
        candidates_1 = prediction_1.iloc[i]['next_item_prediction']
        candidates_2_first = prediction_2.iloc[i]['next_item_prediction'][0:1]
        if candidates_2_first[0] == candidates_1[0] or candidates_2_first[0] == candidates_1[1]:
            continue
        else:
            merge_candidates = np.concatenate([candidates_1[:1], candidates_2_first, candidates_1[1:-1]])
            merged_prediction.iloc[i]['next_item_prediction'] = merge_candidates
    return merged_prediction

In [107]:
merged_prediction = merge_predictions(sasrec_prediction, co_graph_prediction)
merged_prediction.head(5)

100%|██████████| 316971/316971 [01:00<00:00, 5245.10it/s]


,locale,next_item_prediction
0,DE,"[B07LG5T3V9, B099NS1XPG, B099NR3X6D, B08QYYBTM..."
1,DE,"[B004ZXMV4Q, B095TQTZXY, B08BZCKDKQ, B010MJNUZ..."
2,DE,"[B0B5QNFWJ1, B0BBVB89CS, B0BJF4KGCN, B00C3TEAP..."
3,DE,"[3772476953, 3772477917, 395535086X, B0829LZFT..."
4,DE,"[B09J8V9RQQ, B09J8SKX9G, B09J8VPTTW, B09J8SKX9..."


In [108]:
merged_prediction.to_parquet('../predictions/merge_prediction/merge_co_graph_and_sasrec.parquet', engine='pyarrow')

# Statistic between SASRec, co-graph and RetroMAE

In [26]:
valid_sessions_df = read_valid_sessions()
valid_sessions_df.head(10)

,prev_items,next_item,locale
0,['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VS...,B06XG1LZ6Z,UK
1,['B00390YWXE' 'B00390YWXE' 'B09WM9W6WQ'],B01MSUI4FE,JP
2,['B01BM9V6H8' 'B01MG55XDR' 'B07VYSSRL7'],B01M6625ME,UK
3,['B092ZG24S7' 'B09BNHWWZM' 'B08CB1WG5M' '17880...,0241558573,UK
4,['B0B6NY5RM8' 'B09BJGBBBR'],B09BJF6N8K,JP
5,['B0B84YXQLV' 'B09G19MPQH' 'B07GRTY2ZZ' 'B08CB...,B0BD48G63Q,DE
6,['B001O51JCS' 'B001O51JCS'],B0045DNZ9Q,DE
7,['B07V5LWSN5' 'B0BF1CGGSJ' 'B07W4DH45P' 'B08P3...,B07W6J5FB2,UK
8,['B019WI5XXO' 'B08KRZX6DG'],B087BDR5GP,JP
9,['B0874RWZVB' 'B07MP1ZSLJ'],B07Q65PXXP,DE


In [27]:
UK_valid_sessions = valid_sessions_df[valid_sessions_df['locale'] == 'UK']
len(UK_valid_sessions)

130665

In [28]:
sasrec_session_candidates_df = read_sasrec_session_candidates()
sasrec_session_candidates_df.head(10), len(sasrec_session_candidates_df)

(  locale                                         candidates sess_id
 0     UK  [B06XG1LZ6Z, B06XGD9VLV, B076PN1SKG, B01MYUDYP...       0
 1     JP  [B09LCPT9DQ, B09MRYK5CV, B0BB5VQ1L8, B084T9C6W...       1
 2     UK  [B09XBS6WCX, B01EYGW86Y, B01MDOBUCC, B01C5YK17...       2
 3     UK  [0241572614, 1406392979, 024157563X, 024147681...       3
 4     JP  [B0B6PF619D, B0B6P77ZRN, B0B6P2PCMP, B0B6NY4PN...       4
 5     DE  [B0BD48G63Q, B0BD3DGNT9, B0B685KHK6, B0B54PQKG...       5
 6     DE  [B07Q82LRDK, B082XSF8XF, B00H37AVY8, B01EGP3LV...       6
 7     UK  [B013SL2712, B07V5LWSN5, B00CL6353A, B07W6JP97...       7
 8     JP  [B0BFPGHSYX, B09ZTN81QH, B00B57A5IY, B00LE7TO0...       8
 9     DE  [B07MH3K3S8, B07KQHHYQC, B00Z0BTBEA, B07YLZ67Q...       9,
 361581)

In [25]:
sasrec_UK_session_candidates_df = sasrec_session_candidates_df[sasrec_session_candidates_df['locale'] == 'UK']
len(sasrec_UK_session_candidates_df)

130665

In [19]:
co_graph_item_candidates_df = read_co_graph_item_candidates()
co_graph_item_candidates_df.head(10), len(co_graph_item_candidates_df)

(           id                                         candidates
 0  B005ZSSN10  [B07Q1YXVWH, B0B94HRCXK, B09VXRZKWB, B09LCVHCV...
 1  B08PRYN6LD  [B08PSBK59Y, B079RJTR2G, B01N7RTYXE, B08HHD25M...
 2  B09MBZJ48V  [B07VR16HF9, B07FL7GVZG, B09DPR2LZW, B0B9S2FDD...
 3  B08ZN6F26S  [B08PVG787Z, B07PJVGVSM, B08ZN6F26S, 140834646...
 4  B094DGRV7D  [B08KTP6517, B094DGRV7D, B092DFVDC4, B09NXJ9KS...
 5  B09JNNBDH5  [B08K4214KB, B01MG62Z5M, B0963J155G, B008F8PMU...
 6  B08R62WZ1Y  [B09NVJGT2W, B09TB6TK6D, B08NY2NM1M, B08R62WZ1...
 7  B09WK4YNX8  [B0B2X1YX72, B094R3R9XH, B0BDML9477, B0BCPD9YY...
 8  B07KTKFYYS  [B08V1KXBQD, B0B94HRCXK, B07RHT52HX, B0B7J4WSM...
 9  B0B9MPKYJK  [B09HWSDTM3, B09C6H17PK, B08LR3G17D, B09SWTJZH...,
 1410675)

In [20]:
def get_session_last_item(session_df):
    last_items = []
    num_sessions = len(session_df)
    for i in tqdm(range(num_sessions)):
        sess = session_df.iloc[i]
        sess_prev_items = sess['prev_items']
        
        product_list = sess_prev_items.strip('[]').split(' ')
        last_item = product_list[-1].strip("'\n")

        last_items.append(last_item)
    return last_items 

last_items = get_session_last_item(UK_valid_sessions)
last_items[:10]

100%|██████████| 130665/130665 [00:03<00:00, 35654.87it/s]


['B077XGDMD2',
 'B07VYSSRL7',
 '1788009975',
 'B013SL1YXO',
 'B0B7XDNP5W',
 '1405943408',
 'B00ET1X0TO',
 'B08JLWGQVL',
 'B001DQA7GC',
 'B09BPMWGYP']

In [22]:
co_graph_item_candidates_df = co_graph_item_candidates_df.set_index('id', drop=False)
co_graph_item_candidates_df.head(5)

,id,candidates
id,,
B005ZSSN10,B005ZSSN10,"[B07Q1YXVWH, B0B94HRCXK, B09VXRZKWB, B09LCVHCV..."
B08PRYN6LD,B08PRYN6LD,"[B08PSBK59Y, B079RJTR2G, B01N7RTYXE, B08HHD25M..."
B09MBZJ48V,B09MBZJ48V,"[B07VR16HF9, B07FL7GVZG, B09DPR2LZW, B0B9S2FDD..."
B08ZN6F26S,B08ZN6F26S,"[B08PVG787Z, B07PJVGVSM, B08ZN6F26S, 140834646..."
B094DGRV7D,B094DGRV7D,"[B08KTP6517, B094DGRV7D, B092DFVDC4, B09NXJ9KS..."


In [24]:
co_graph_UK_session_candidates_df = co_graph_item_candidates_df.loc[last_items]
co_graph_UK_session_candidates_df[:10], len(co_graph_UK_session_candidates_df)

(                    id                                         candidates
 id                                                                       
 B077XGDMD2  B077XGDMD2  [B077XGDMD2, B06XGDZVZR, B06XG1LZ6Z, B076PN1SK...
 B07VYSSRL7  B07VYSSRL7  [B09XBS6WCX, B01M6625ME, B094W9KT4Y, B09SHSZ2C...
 1788009975  1788009975  [1839941960, 1788009975, 0241563437, 000853222...
 B013SL1YXO  B013SL1YXO  [B013SL1YXO, B08FBZ8QSL, B085WYXXLF, B087MV18C...
 B0B7XDNP5W  B0B7XDNP5W  [B09N959VPS, B096XR7M84, B077P3D31Q, B09BZR9JF...
 1405943408  1405943408  [1472271246, B09RGP9NDZ, 1405951184, 152934241...
 B00ET1X0TO  B00ET1X0TO  [B09MP8BCYP, B07MCFD1MK, B088FSHMQ3, B00LDSN71...
 B08JLWGQVL  B08JLWGQVL  [B07MSJCPR4, B0761P8SDY, B07QZ9NN13, B08PPDTH9...
 B001DQA7GC  B001DQA7GC  [B001DQA7GC, B0B6Y61879, B07LFK5LNS, B08GBX7ZB...
 B09BPMWGYP  B09BPMWGYP  [B09BPV8Q2X, B09BPWHRYQ, B089RQGCJ9, B07Q5BGKV...,
 130665)

In [37]:
retromae_UK_session_candidates_df = read_retromae_session_candidates()
retromae_UK_session_candidates_df.head(5), len(retromae_UK_session_candidates_df)

(  locale                               next_item_prediction
 0     UK  [B06XGD9VLV, B06XG1LZ6Z, B06XGDZVZR, B0BJQ95XW...
 1     UK  [B00JLH125A, B09GGGFKFR, B015DVC6G6, B017A0OY8...
 2     UK  [1406357383, 1406363073, B0BGY899Q5, B074G22VZ...
 3     UK  [B013SL2712, B07W6JP976, B07W6JJ253, B07W7LHVB...
 4     UK  [B0B7XCS3QT, B077P3D31Q, B098JJLSWX, B08PYWFGB...,
 130665)

In [41]:
# hit or
def cal_or_hit_3(ground_truth_list, candidates_list_1, candidates_list_2, candidates_list_3):
    hits = []
    for i in tqdm(range(len(ground_truth_list))):
        ground_truth = ground_truth_list.iloc[i]
        candidates_1 = candidates_list_1.iloc[i]
        candidates_2 = candidates_list_2.iloc[i]
        candidates_3 = candidates_list_3.iloc[i]
        hit = 0.0
        if (ground_truth in candidates_1) or (ground_truth in candidates_2) or (ground_truth in candidates_3):
            hit = 1.0
        hits.append(hit)
    return np.array(hits).mean()

In [42]:
# hit concurrently
def cal_co_hit_3(ground_truth_list, candidates_list_1, candidates_list_2, candidates_list_3):
    hits = []
    for i in tqdm(range(len(ground_truth_list))):
        ground_truth = ground_truth_list.iloc[i]
        candidates_1 = candidates_list_1.iloc[i]
        candidates_2 = candidates_list_2.iloc[i]
        candidates_3 = candidates_list_3.iloc[i]
        hit = 0.0
        if (ground_truth in candidates_1) and (ground_truth in candidates_2) and (ground_truth in candidates_3):
            hit = 1.0
        hits.append(hit)
    return np.array(hits).mean()

In [33]:
def cal_hit_and_mrr(ground_truth_list, candidates_list):
    hits, mrrs = [], []
    for i in tqdm(range(len(ground_truth_list))):
        ground_truth = ground_truth_list.iloc[i]
        candidates = candidates_list.iloc[i]
        hit, mrr = 0.0, 0.0
        for j in range(len(candidates)):
            if ground_truth == candidates[j]:
                hit = 1.0
                mrr = 1.0 / (j + 1)
                break
        hits.append(hit)
        mrrs.append(mrr)
    return np.array(hits).mean(), np.array(mrrs).mean()

In [35]:
cal_hit_and_mrr(UK_valid_sessions['next_item'], sasrec_UK_session_candidates_df['candidates'])

100%|██████████| 130665/130665 [00:04<00:00, 26841.49it/s]


(0.6061454865495733, 0.26247991444563873)

In [36]:
cal_hit_and_mrr(UK_valid_sessions['next_item'], co_graph_UK_session_candidates_df['candidates'])

100%|██████████| 130665/130665 [00:03<00:00, 36766.50it/s]


(0.5390425898289519, 0.26683356338399805)

In [38]:
cal_hit_and_mrr(UK_valid_sessions['next_item'], retromae_UK_session_candidates_df['next_item_prediction'])

100%|██████████| 130665/130665 [00:03<00:00, 40035.24it/s]


(0.7314047373053227, 0.24190486057082744)

In [39]:
cal_co_hit_3(UK_valid_sessions['next_item'], sasrec_UK_session_candidates_df['candidates'], co_graph_UK_session_candidates_df['candidates'], 
           retromae_UK_session_candidates_df['next_item_prediction'])

100%|██████████| 130665/130665 [00:09<00:00, 13700.01it/s]


0.4064363065855432

In [45]:
cal_or_hit_3(UK_valid_sessions['next_item'], sasrec_UK_session_candidates_df['candidates'], co_graph_UK_session_candidates_df['candidates'], 
           retromae_UK_session_candidates_df['next_item_prediction'])

100%|██████████| 130665/130665 [00:08<00:00, 15313.12it/s]


0.8306202885240883

In [48]:
# 300 
cal_or_hit_3(UK_valid_sessions['next_item'],
            sasrec_UK_session_candidates_df['candidates'].apply(lambda x : x[:100]), 
            co_graph_UK_session_candidates_df['candidates'].apply(lambda x : x[:100]), 
            retromae_UK_session_candidates_df['next_item_prediction'].apply(lambda x : x[:100]))

100%|██████████| 130665/130665 [00:06<00:00, 21565.72it/s]


0.7884590364673019

In [49]:
# 450
cal_or_hit_3(UK_valid_sessions['next_item'],
            sasrec_UK_session_candidates_df['candidates'].apply(lambda x : x[:150]), 
            co_graph_UK_session_candidates_df['candidates'].apply(lambda x : x[:150]), 
            retromae_UK_session_candidates_df['next_item_prediction'].apply(lambda x : x[:150]))

100%|██████████| 130665/130665 [00:05<00:00, 21875.40it/s]


0.8063750813148127

In [50]:
# 600
cal_or_hit_3(UK_valid_sessions['next_item'],
            sasrec_UK_session_candidates_df['candidates'].apply(lambda x : x[:200]), 
            co_graph_UK_session_candidates_df['candidates'].apply(lambda x : x[:200]), 
            retromae_UK_session_candidates_df['next_item_prediction'].apply(lambda x : x[:200]))

100%|██████████| 130665/130665 [00:07<00:00, 17533.22it/s]


0.8169670531511881

In [46]:
cal_or_hit(UK_valid_sessions['next_item'], co_graph_UK_session_candidates_df['candidates'], retromae_UK_session_candidates_df['next_item_prediction'])

100%|██████████| 130665/130665 [00:05<00:00, 22998.71it/s]


0.817257873187158

# Statistic between SASRec, co-graph and XLM-RoBerta

In [4]:
def cal_hit_and_mrr(ground_truth_list, candidates_list):
    hits, mrrs = [], []
    for i in tqdm(range(len(ground_truth_list))):
        ground_truth = ground_truth_list.iloc[i]
        candidates = candidates_list.iloc[i]
        hit, mrr = 0.0, 0.0
        for j in range(len(candidates)):
            if ground_truth == candidates[j]:
                hit = 1.0
                mrr = 1.0 / (j + 1)
                break
        hits.append(hit)
        mrrs.append(mrr)
    return np.array(hits).mean(), np.array(mrrs).mean()

In [5]:
valid_sessions_df = read_valid_sessions()
valid_sessions_df.head(10)

,prev_items,next_item,locale
0,['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VS...,B06XG1LZ6Z,UK
1,['B00390YWXE' 'B00390YWXE' 'B09WM9W6WQ'],B01MSUI4FE,JP
2,['B01BM9V6H8' 'B01MG55XDR' 'B07VYSSRL7'],B01M6625ME,UK
3,['B092ZG24S7' 'B09BNHWWZM' 'B08CB1WG5M' '17880...,0241558573,UK
4,['B0B6NY5RM8' 'B09BJGBBBR'],B09BJF6N8K,JP
5,['B0B84YXQLV' 'B09G19MPQH' 'B07GRTY2ZZ' 'B08CB...,B0BD48G63Q,DE
6,['B001O51JCS' 'B001O51JCS'],B0045DNZ9Q,DE
7,['B07V5LWSN5' 'B0BF1CGGSJ' 'B07W4DH45P' 'B08P3...,B07W6J5FB2,UK
8,['B019WI5XXO' 'B08KRZX6DG'],B087BDR5GP,JP
9,['B0874RWZVB' 'B07MP1ZSLJ'],B07Q65PXXP,DE


In [6]:
sasrec_session_candidates_df = read_sasrec_session_candidates()
sasrec_session_candidates_df.head(10), len(sasrec_session_candidates_df), len(sasrec_session_candidates_df.iloc[0]['candidates'])

(  locale                                         candidates sess_id
 0     UK  [B06XG1LZ6Z, B06XGD9VLV, B076PN1SKG, B01MYUDYP...       0
 1     JP  [B09LCPT9DQ, B09MRYK5CV, B0BB5VQ1L8, B084T9C6W...       1
 2     UK  [B09XBS6WCX, B01EYGW86Y, B01MDOBUCC, B01C5YK17...       2
 3     UK  [0241572614, 1406392979, 024157563X, 024147681...       3
 4     JP  [B0B6PF619D, B0B6P77ZRN, B0B6P2PCMP, B0B6NY4PN...       4
 5     DE  [B0BD48G63Q, B0BD3DGNT9, B0B685KHK6, B0B54PQKG...       5
 6     DE  [B07Q82LRDK, B082XSF8XF, B00H37AVY8, B01EGP3LV...       6
 7     UK  [B013SL2712, B07V5LWSN5, B00CL6353A, B07W6JP97...       7
 8     JP  [B0BFPGHSYX, B09ZTN81QH, B00B57A5IY, B00LE7TO0...       8
 9     DE  [B07MH3K3S8, B07KQHHYQC, B00Z0BTBEA, B07YLZ67Q...       9,
 361581,
 300)

In [7]:
co_graph_item_candidates_df = read_co_graph_item_candidates()
co_graph_item_candidates_df.head(10), len(co_graph_item_candidates_df), len(co_graph_item_candidates_df.iloc[0]['candidates'])

(           id                                         candidates
 0  B005ZSSN10  [B07Q1YXVWH, B0B94HRCXK, B09VXRZKWB, B09LCVHCV...
 1  B08PRYN6LD  [B08PSBK59Y, B079RJTR2G, B01N7RTYXE, B08HHD25M...
 2  B09MBZJ48V  [B07VR16HF9, B07FL7GVZG, B09DPR2LZW, B0B9S2FDD...
 3  B08ZN6F26S  [B08PVG787Z, B07PJVGVSM, B08ZN6F26S, 140834646...
 4  B094DGRV7D  [B08KTP6517, B094DGRV7D, B092DFVDC4, B09NXJ9KS...
 5  B09JNNBDH5  [B08K4214KB, B01MG62Z5M, B0963J155G, B008F8PMU...
 6  B08R62WZ1Y  [B09NVJGT2W, B09TB6TK6D, B08NY2NM1M, B08R62WZ1...
 7  B09WK4YNX8  [B0B2X1YX72, B094R3R9XH, B0BDML9477, B0BCPD9YY...
 8  B07KTKFYYS  [B08V1KXBQD, B0B94HRCXK, B07RHT52HX, B0B7J4WSM...
 9  B0B9MPKYJK  [B09HWSDTM3, B09C6H17PK, B08LR3G17D, B09SWTJZH...,
 1410675,
 300)

In [8]:
co_graph_item_candidates_df = co_graph_item_candidates_df.set_index('id', drop=False)
co_graph_item_candidates_df.head(5)

,id,candidates
id,,
B005ZSSN10,B005ZSSN10,"[B07Q1YXVWH, B0B94HRCXK, B09VXRZKWB, B09LCVHCV..."
B08PRYN6LD,B08PRYN6LD,"[B08PSBK59Y, B079RJTR2G, B01N7RTYXE, B08HHD25M..."
B09MBZJ48V,B09MBZJ48V,"[B07VR16HF9, B07FL7GVZG, B09DPR2LZW, B0B9S2FDD..."
B08ZN6F26S,B08ZN6F26S,"[B08PVG787Z, B07PJVGVSM, B08ZN6F26S, 140834646..."
B094DGRV7D,B094DGRV7D,"[B08KTP6517, B094DGRV7D, B092DFVDC4, B09NXJ9KS..."


In [12]:
last_items = get_session_last_item(valid_sessions_df)
last_items[:10]

100%|██████████| 361581/361581 [00:09<00:00, 38664.68it/s]


['B077XGDMD2',
 'B09WM9W6WQ',
 'B07VYSSRL7',
 '1788009975',
 'B09BJGBBBR',
 'B0BD489B72',
 'B001O51JCS',
 'B013SL1YXO',
 'B08KRZX6DG',
 'B07MP1ZSLJ']

In [15]:
co_graph_session_candidates_df = co_graph_item_candidates_df.loc[last_items]
co_graph_session_candidates_df[:10], len(co_graph_session_candidates_df), len(co_graph_session_candidates_df.iloc[0]['candidates'])

(                    id                                         candidates
 id                                                                       
 B077XGDMD2  B077XGDMD2  [B077XGDMD2, B06XGDZVZR, B06XG1LZ6Z, B076PN1SK...
 B09WM9W6WQ  B09WM9W6WQ  [B09LCPT9DQ, B09WM9W6WQ, B092D5HM5S, B09MRYK5C...
 B07VYSSRL7  B07VYSSRL7  [B09XBS6WCX, B01M6625ME, B094W9KT4Y, B09SHSZ2C...
 1788009975  1788009975  [1839941960, 1788009975, 0241563437, 000853222...
 B09BJGBBBR  B09BJGBBBR  [B0B6PF619D, B0B6NRKVZF, B0B6NY4PNC, B0B6P77ZR...
 B0BD489B72  B0BD489B72  [B0BD48G63Q, B0B6ZNHCVV, B0B685KHK6, B0B7RNG4X...
 B001O51JCS  B001O51JCS  [B07Q82LRDK, B08BR288Z7, B09MP8BCYP, B01NCX3W3...
 B013SL1YXO  B013SL1YXO  [B013SL1YXO, B08FBZ8QSL, B085WYXXLF, B087MV18C...
 B08KRZX6DG  B08KRZX6DG  [B00LE7TO0K, B00B57A5IY, B08KRZX6DG, B086LGXKY...
 B07MP1ZSLJ  B07MP1ZSLJ  [B07MH3K3S8, B081GT5MW8, B07Q65PXXP, B0041GWW2...,
 361581,
 300)

In [14]:
roberta_session_candidates_df = read_roberta_session_candidates()
roberta_session_candidates_df.head(10), len(roberta_session_candidates_df), len(roberta_session_candidates_df.iloc[0]['next_item_prediction'])

(                                next_item_prediction  \
 0  [B096ZT4DK4, B09XCGNN6F, B09XCHDLYR, B06XGD9VL...   
 1  [B08ZHJKF28, B09LCPT9DQ, B09MRYK5CV, B09G9YRX1...   
 2  [B06ZYHMRST, B01017JLP6, B007CW985C, B0017RTPK...   
 3  [B01M5EBAO4, B07TYFS1K7, B08BX71RLT, B0BGY899Q...   
 4  [B0B6NXJMJJ, B0B6PF619D, B0B6P77ZRN, B0B6NRKVZ...   
 5  [B0BD48G63Q, B0BD3DGNT9, B0BD3FYDPB, B09BJNXGP...   
 6  [B00GYSC172, B088NQ34PW, B06ZZ6ZCD6, B07CNV566...   
 7  [B07V3HG2YD, B0B8NKMT5S, B07XRFMG5B, B019O1R4D...   
 8  [B0BFPGHSYX, B005A0PEQ0, B082FHGR57, B0152C7KP...   
 9  [B096DSVNSN, B096D4C8HT, B07MH3K3S8, B00Z0BTBE...   
 
                                               scores  
 0  [268.6368103027344, 268.5672607421875, 268.567...  
 1  [269.99102783203125, 269.4064636230469, 269.30...  
 2  [264.7541198730469, 264.58624267578125, 264.49...  
 3  [267.71771240234375, 266.8636779785156, 266.69...  
 4  [269.4986572265625, 269.49267578125, 269.47979...  
 5  [268.35125732421875, 267.995635

In [16]:
# hit or
def cal_or_hit_3(ground_truth_list, candidates_list_1, candidates_list_2, candidates_list_3):
    hits = []
    for i in tqdm(range(len(ground_truth_list))):
        ground_truth = ground_truth_list.iloc[i]
        candidates_1 = candidates_list_1.iloc[i]
        candidates_2 = candidates_list_2.iloc[i]
        candidates_3 = candidates_list_3.iloc[i]
        hit = 0.0
        if (ground_truth in candidates_1) or (ground_truth in candidates_2) or (ground_truth in candidates_3):
            hit = 1.0
        hits.append(hit)
    return np.array(hits).mean()

In [17]:
# 300 
cal_or_hit_3(valid_sessions_df['next_item'],
            sasrec_session_candidates_df['candidates'].apply(lambda x : x[:100]), 
            co_graph_session_candidates_df['candidates'].apply(lambda x : x[:100]), 
            roberta_session_candidates_df['next_item_prediction'].apply(lambda x : x[:100]))

100%|██████████| 361581/361581 [00:15<00:00, 23992.36it/s]


0.7845710919544998

In [18]:
# 450
cal_or_hit_3(valid_sessions_df['next_item'],
            sasrec_session_candidates_df['candidates'].apply(lambda x : x[:150]), 
            co_graph_session_candidates_df['candidates'].apply(lambda x : x[:150]), 
            roberta_session_candidates_df['next_item_prediction'].apply(lambda x : x[:150]))

100%|██████████| 361581/361581 [00:15<00:00, 23325.25it/s]


0.8025449346066303

In [19]:
# 600
cal_or_hit_3(valid_sessions_df['next_item'],
            sasrec_session_candidates_df['candidates'].apply(lambda x : x[:200]), 
            co_graph_session_candidates_df['candidates'].apply(lambda x : x[:200]), 
            roberta_session_candidates_df['next_item_prediction'].apply(lambda x : x[:200]))

100%|██████████| 361581/361581 [00:16<00:00, 21450.73it/s]


0.8133668527937032

# Feature statistic between ground truth and candidates 

In [56]:
pd.set_option('display.max_colwidth', 2000)

In [25]:
valid_sessions_df = read_valid_sessions()
valid_sessions_df.head(5)

,prev_items,next_item,locale
0,['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VS...,B06XG1LZ6Z,UK
1,['B00390YWXE' 'B00390YWXE' 'B09WM9W6WQ'],B01MSUI4FE,JP
2,['B01BM9V6H8' 'B01MG55XDR' 'B07VYSSRL7'],B01M6625ME,UK
3,['B092ZG24S7' 'B09BNHWWZM' 'B08CB1WG5M' '17880...,0241558573,UK
4,['B0B6NY5RM8' 'B09BJGBBBR'],B09BJF6N8K,JP


In [15]:
product_data = read_product_data()
product_data = product_data.set_index('id')
product_data.head(5)

,locale,title,price,brand,color,size,model,material,author,desc
id,,,,,,,,,,
B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück


In [26]:
valid_UK_sessions_df = valid_sessions_df[valid_sessions_df['locale'] == 'UK']
valid_UK_sessions_df.head(5)

,prev_items,next_item,locale
0,['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VS...,B06XG1LZ6Z,UK
2,['B01BM9V6H8' 'B01MG55XDR' 'B07VYSSRL7'],B01M6625ME,UK
3,['B092ZG24S7' 'B09BNHWWZM' 'B08CB1WG5M' '17880...,0241558573,UK
7,['B07V5LWSN5' 'B0BF1CGGSJ' 'B07W4DH45P' 'B08P3...,B07W6J5FB2,UK
10,['B07P4CYP4S' 'B07P4CYP4S' 'B0B7XDNP5W'],B0B7XCS3QT,UK


In [27]:
valid_UK_sessions_df['prev_items'] = valid_UK_sessions_df['prev_items'].apply(lambda x : eval(x.replace(' ', ',')))
valid_UK_sessions_df.head(5)

/tmp/ipykernel_79588/748372954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_UK_sessions_df['prev_items'] = valid_UK_sessions_df['prev_items'].apply(lambda x : eval(x.replace(' ', ',')))


,prev_items,next_item,locale
0,"[B09VSN9GLS, B09VSG9DCG, B0BJ5L1ZPH, B09VSN9GL...",B06XG1LZ6Z,UK
2,"[B01BM9V6H8, B01MG55XDR, B07VYSSRL7]",B01M6625ME,UK
3,"[B092ZG24S7, B09BNHWWZM, B08CB1WG5M, 178800997...",0241558573,UK
7,"[B07V5LWSN5, B0BF1CGGSJ, B07W4DH45P, B08P395NW...",B07W6J5FB2,UK
10,"[B07P4CYP4S, B07P4CYP4S, B0B7XDNP5W]",B0B7XCS3QT,UK


In [80]:
sasrec_session_candidates_df = read_sasrec_session_candidates()
sasrec_UK_session_candidates_df = sasrec_session_candidates_df[sasrec_session_candidates_df['locale'] == 'UK']
sasrec_UK_session_candidates_df.head(5)

,locale,candidates,sess_id
0,UK,"[B06XG1LZ6Z, B06XGD9VLV, B076PN1SKG, B01MYUDYP7, B06XGDZVZR, B07ZZ4QVYT, B097PVPKV4, B087F749NB, B017MOYTQC, B09FY916F1, B08GK6HD71, B077VX8Q12, B07449Q2C1, B076B5N488, B094JZL492, B08147PN1V, B09TW5134P, B076PJWH6J, B093SZNPZN, B087DYVDXL, B01F1R1LFY, B00I0UKKD4, B09XDVQFWD, B09WVLM2TB, B083QYDFYC, B01LX7M2J1, B09NQ1GK4P, B0B2N3RGMF, B08G86V98M, B09W1V6Q3M, B072HCH1R2, B08F7P8R6D, B07L3L4PQH, B084ZQ2NW8, B0B9BKBCLC, B07L3LHR9K, B07RMJ5X1R, B08NCYBSJB, B097Y7S2KY, B0BGKQDMJL, B07BK7HMN2, B0769VZRLN, B08ND4B873, B076H815PD, B09P54SPTQ, B09FY7VX9S, B07F3X5CFS, B08SHJVJZ6, B0BGKM19NY, B000V599Y2, B097Y7MDHN, B08ND2PCQV, B09XQSP9LD, B07F2VQQ4G, B09WF4MM8F, B07L3LJHVC, B071XXBXDT, B07RNZZ89K, B098QH6122, B09BVVT6BL, B077VXHCCH, B0728GGX6Y, B0B5Y4TXMZ, B0843TL5YY, B098QF9QPP, B077VWJYHT, B086TSN5SM, B077VXNL67, B09TWBBFY9, B01MPYILA7, B0B2JT2BND, B09NPZPKYQ, B08147T2RH, B09FK17XPN, B096KPBTR8, B0BFPKSQ7X, B09WD5ZKWT, B01GUHTFL6, B08G8BPZF4, B084Q5NX83, B077VSMJ8R, B0B5Y2WMP5, B083WNMZPZ, B09XCHDLYR, B0856KQ4D2, B08BNH48JM, B07448LCWC, B077VXNL6H, B06ZYNB1SK, B08CZRT585, B08427M62D, B08MWRW6MZ, B084383K5H, B09FY7XYJ6, B07RK95WGT, B087FB54VN, B09V59XRQ5, B07SNHRTSW, B06WP1TV3K, B09XQSHKGL, ...]",0
2,UK,"[B09XBS6WCX, B01EYGW86Y, B01MDOBUCC, B01C5YK17K, B08FMKZFV2, B0B3ZCMTRP, B07ZL7BPD5, B093BPPXTM, B07CFSC84Q, B000ZT07EK, B01M8I1WX4, B08RLHG74X, B01LZGCRBJ, B01NCB79N3, B01IT3KKNQ, B08M1DXCLX, B01LF3VCYI, B01M6625ME, B07RXKJML5, B0BG59CHF3, B01IB2S9II, B07RZL269R, B01EYKV5LO, B08JCNZVSR, B01MQ62UJV, B075HH6L2B, B07FQC3B57, B07GJS6QNP, B0816QN7ZJ, B01N4NOVAW, B01CK0P9B2, B0992289FM, B079WKK1B2, B081NB2T54, B017QJ4S6E, B06X3YQCC2, B09Q98RMRZ, B01CNSNSPA, B00HP48OCU, B00L529BAC, B0954883CN, B01EYGXQ0Q, B07L3GXSZ6, B071DZFL6F, B0B6CW27DR, B0B4DXPHJ5, B09XBQ6H8W, B07GBCWD74, B09X5XLZR9, B0001P03O2, B07T3XCZ8F, B097YSXMMZ, B0043A4IVU, B005NY0Q6Q, B08F2SLK68, B09P7FGC62, B07FT1CR5P, B01DVQ13QE, B000WGYA58, B01M2Z8J8C, B091G5ZMBR, B01FSW591O, B01IB2RZV0, B07D3GG47V, B09Q8PTQ33, B07GWL1LXJ, B0866F1RDW, B089GHMK6W, B083DTS39G, B09C885NZH, B06WRN4MZZ, B0BDFXKGST, B004L8O4CC, B0BG59YGR3, B01LPOS1W8, B01M6628A9, B008HKMK8U, B01EYGY1N2, B01EYKV5U0, B01M1KZ73L, B09B479RY9, B09D9RZDMK, B091Q1HXSN, B08Z7HDD4F, B01IT3KVB2, B00AP7TU2C, B01LF3VD38, B08TMMXWN7, B0922GQHD4, B087Q8GBST, B09YPW4NFZ, B00PC5Q1I8, B0BC476W3L, B015Z9SAES, B07GQLQ3FP, B0B2BW17YJ, B07ZX5WMY7, B00AMZSECE, B016O1QOTE, B01M4L4UB6, ...]",2
3,UK,"[0241572614, 1406392979, 024157563X, 024147681X, 0008544840, 140528272X, 0008532222, 024154050X, 1408346486, 1529083346, 0241563437, 0241457920, 1529023866, 1529046483, 0140565817, 024135739X, 1839945931, 1788004671, 1801314918, 0141376139, 1509801251, 1405950730, 0241547164, 1529025346, 152906421X, 1471167984, 0241488834, 072327584X, 0241466636, 0141376104, 1405948752, 0241484340, 1509862668, 1509804773, 0241539749, 0241548349, 1509866949, 1474969275, 0241287790, 1474964419, 0008560749, 0241466563, 1529087058, 1529050715, 014133357X, 1408356163, 0230756050, 024138673X, 1509804781, 0141373342, 1529048494, 0241357217, 000847902X, 0241392403, 1780556454, 0007413572, 1405296895, 150983043X, 0007215991, 0008172714, 0241586577, 1407184636, 0008305757, 1526382423, 1801313652, 1529034396, 1955421765, 1526632217, 1803700890, 0007513763, 0746070055, 0241595797, B07X34VJT7, 1788003675, 0007581866, 1509866957, 1509806970, 1407170708, 0241330556, 1529034361, 1406363073, 1408354403, 1408331608, 1800783795, 1913944166, B0BCNX1GCV, 0241466601, 0007977174, 1529023602, 0723263663, 1447277260, 0723287120, 0702318000, 1407170767, B0875SQT57, B0BGN68L2J, B07YGPB9JY, 1529074932, 1408356155, 0008131813, ...]",3
7,UK,"[B013SL2712, B07V5LWSN5, B00CL6353A, B07W6JP976, B07W6GGC8W, B07W4DHYR9, B09WDJNBT1, B01GV6HP0E, B08FBZ8QSL, B07W6HJVDM, B07W6JJ253, B08W55GSTV, B015LSEUS8, B09WDLRHRN, B00M75WPKO, B07Q93D73F, B09G344JRY, B00XJ3LVPI, B0B27NSRJB, B00OK49SMI, B073TSKFRX, B0B3DZJLJB, B07W6HXZ5J, B07W7LHVBM, B07W7KFLBN, B087JC9F8R, B085RGMMV

In [76]:
retromae_UK_session_candidates_df = read_retromae_session_candidates()
retromae_UK_session_candidates_df.head(5)

,locale,next_item_prediction
0,UK,"[B06XGD9VLV, B06XG1LZ6Z, B06XGDZVZR, B0BJQ95XWV, B09WVLM2TB, B09XDVQFWD, B0BJ66G5LN, B0B2N3RGMF, B09BD65VSF, B09NPZPKYQ, B09NQ1GK4P, B08BNH48JM, B09DXD4STW, B087FB54VN, B087DYVDXL, B087F749NB, B098QH6122, B09P54SPTQ, B09P52T43Z, B095NZWZDH, B07FCBV2Q5, B077VSMJ8R, B0B2JT2BND, B08F7P8R6D, B0B2JSKN7X, B01HF2A8PC, B097DM36WD, B0B9BKBCLC, B084ZQ5JF4, B09TP6PWNB, B09SLBY68P, B077VXHCCH, B07BK7HMN2, B09TW5134P, B085Y1L7LX, B085Y1DP3P, B07G5ZSNY2, B08VN7FMLT, B09WM8JX9T, B07BK7739T, B07BK7HZ2L, B09XQSP9LD, B09T6QTGC8, B097Y7S2KY, B097Y7MDHN, B084ZQ2NW8, B09W1V6Q3M, B098QF9QPP, B00I0UKKD4, B09TZS4S2Z, B07TH98T68, B07TGBMC72, B084ZPN91X, B092ZQDZF5, B07TBQB8FK, B09TWBBFY9, B09FY916F1, B077VXNL6H, B083QXSY66, B084QF8P74, B079BL4CPZ, B01MPYILA7, B01MYUDYP7, B07SNHRTSW, B07WQ4B3WZ, B097PVPKV4, B09FY7VX9S, B09YXB4JKW, B077VX8Q12, B082L9D9R4, B09SPZ3RT1, B076PN1SKG, B084ZPL21L, B077VXNL67, B077VWJYHT, B07RNZZ89K, B07RMJ5X1R, B07RK95WGT, B07T8FX6L2, B072HCH1R2, B0728GGX6Y, B071XXBXDT, B01MZXG8J0, B09TXNC75K, B017MOYTQC, B096ZT4DK4, B083QYDFYC, B09TW5LJ33, B096KPBTR8, B09HW2D1M9, B09YXQ313K, B09FY7XYJ6, B015Q0MXM6, B073W395GR, B07ZZ4QVYT, B07T8GJRH4, B09SPY1SVZ, B09FY6G9NW, B07T9QHGY9, B0BGKM19NY, ...]"
1,UK,"[B00JLH125A, B09GGGFKFR, B015DVC6G6, B017A0OY86, B00KP0UETW, B06XX9W93F, B00L17OXRM, B0B6CTFYX6, B01GJ6ICYO, B07VS2H3P6, B01M8KBTGW, B09X5XLZR9, B0BGSX7DL5, B07BMG1S8X, B01IT3KVB2, B09XBQ8XSK, B01M8I1WX4, B01MG1CO07, B01LPOSCBI, B01CDUR2S2, B07GJS6QNP, B01MDOBUCC, B079WKK1B2, B0BG647MF9, B0B1D4665T, B07D3GG47V, B0B6CW5HCN, B09GB1312D, B09QJ2N5C2, B0744HJQK9, B091G673KC, B01F02DYLY, B01MQ62UJV, B081NBB59N, B00HP48OCU, B096YFM2K5, B01MAYMOCG, B01CNSNSPA, B01CCZ4LNM, B09XBQ5DSZ, B091G35Q5T, B01LF3WD6O, B091G67DJ9, B00M4GYASG, B01LPOS1W8, B01M7ULTTN, B07115S814, B09TRRX7XT, B01N20N3FB, B0BB3F54F5, B0B1QK35XL, B094DMFLC6, B07GJS6QPF, B00CD31U24, B08SK6SPCP, B00FQV015C, B01JZTY29Q, B08DJR5NWN, B09P1PFV3H, B01N2G0TLF, B01IT3KKNQ, B01CJSI624, B01MU1PF62, B08F2Q6FRX, B00H0GFY3K, B01GJ6ICVC, B01AWH5AW8, B08FMKZFV2, B073PGNDTP, B0B6CW77DS, B01EYKV5U0, B01GRWA4F0, B01AWH4JFC, B01IB2SSA2, B01IT3KURW, B0BF62Q3Z7, B01BNLTTRE, B01NCB79N3, B0BC9SBPN8, B00JLH126Y, B07CT2JP2L, B075VVPVSL, B01CNSNSHI, B07TVRQYCK, B01IBO19V0, B00DB5QTQ0, B01FHHQBEY, B094DM177F, B01LVUOY49, B01017JLP6, B07WSQK12N, B06XYHFK72, B06ZYHMRST, B07H4X1YJC, B01LF3VHWK, B0B6W7VVMF, B07ZL7BPD5, B0B1F4L96V, B079YGBX3V, B01LQ6EWQ4, ...]"
2,UK,"[1406357383, 1406363073, B0BGY899Q5, B074G22VZC, 0008544840, 0008305749, B01M5EBAO4, B07G4MNXPZ, B0BGF31237, 0439873770, B00E32X422, 0552545104, B09ZTG7344, 1405296895, B07XYYX7ZM, B09GXHNH7F, 1801313652, B0BJ54PZV6, B095LR178T, 140528272X, B0154LBMFG, B08D6DD4NG, B0BC1FTLZS, 1405290528, B0B7L5LPLW, 0593125177, 024135739X, 000847902X, 1408354403, 1841215651, 184869069X, B09XB5G823, B0B6F3FGQZ, B09ZTGMS34, B09ZSMGFMT, B098NHBNR6, B07W5V4Q6B, B097H5X5M1, B0B8DWYPK6, 0803738412, 1509804773, B00IMKXIBI, 150983043X, B07X9LJST9, 1529025346, 0241456541, B07HMVLQB2, 0141376147, 0141376139, B009S8AZ64, B08DP195R4, B095YMFCGR, 0241357349, 0241357306, B097JZ86B5, 1787009726, 1803700890, B08CL1L5M3, B097JK5BSK, 1407170740, B09BL1NLS1, B09734YB3P, B07X95M3RV, B0BBXLD735, B08FJ29PYV, B09C1WPN4K, B094R357SS, B09F3Q6MNZ, B09BQT82M3, 1801314918, 152906421X, B0B778ZK32, B08HD2BDLP, B07GY2JB9H, B08BVXRTMM, B094QTHLMH, B09DN1FKN1, B09BVWGLDY, 0008560749, B09BVWTKM6, B087MVYZ3C, B0B1ZM95GZ, B0933FSJRZ, B077G5P7NS, 1408849828, 1526620197, 0241466636, B0B4P1KJ42, 1529069793, B0894NM4VP, 0241558573, 1787835901, B0B4D5M6QY, B09B9PY6TC, B092LT7CXT, B0BBS7BB3Q, B09BQXMSK8, 1529087058, 1529023866, B0BCZGMYQV, ...]"
3,UK,"[B013SL2712, B07W6JP976, B07W6JJ253, B07W7LHVBM, B07W4DHYR9, B07W7L11LJ, B013SL2050, B01GV6HP0E, B013SL26LS, B07W4DHYLC, B002MMY4WY, B004N627KS, B0BF1D5W4D, B00A389JEW, B08FBZW5QD, B08FBZ8QSL, B08FBZVDM5, B00XJ3LVPI, B0957ZDFJ2, B0074486GA, B0055JDM06, B07K2PK3BV, B08HKYNY73, B0B8C4Z38P, B09NXMPRGG, B016NY7784, B07W7K1LRV, B

In [109]:
def compare(index):
    sess = valid_UK_sessions_df.iloc[index]
    print('***********prev items**********')
    prev_items = sess['prev_items'][-10:]
    for prev_item in prev_items:
        prev_item = product_data.loc[prev_item]
        print(prev_item)

    last_item = sess['prev_items'][-1]
    last_item = product_data.loc[last_item]
    print(last_item)
    print('***********next item**********')
    next_item = sess['next_item']
    next_item = product_data.loc[next_item]
    print(next_item)
    print('***********candidates**********')
    candidates = sasrec_UK_session_candidates_df.iloc[index]['candidates']
    for i in range(15):
        candidate = candidates[i]
        candidate = product_data.loc[candidate]
        print(candidate)

In [110]:
def compare_retromae(index):
    sess = valid_UK_sessions_df.iloc[index]
    print('***********prev items**********')
    prev_items = sess['prev_items'][-10:]
    for prev_item in prev_items:
        prev_item = product_data.loc[prev_item]
        print(prev_item)

    last_item = sess['prev_items'][-1]
    last_item = product_data.loc[last_item]
    print(last_item)
    print('***********next item**********')
    next_item = sess['next_item']
    next_item = product_data.loc[next_item]
    print(next_item)
    print('***********candidates**********')
    candidates = retromae_UK_session_candidates_df.iloc[index]['next_item_prediction']
    for i in range(15):
        candidate = candidates[i]
        candidate = product_data.loc[candidate]
        print(candidate)

In [41]:
def cal_hit_and_mrr(ground_truth_list, candidates_list):
    hits, mrrs = [], []
    for i in tqdm(range(len(ground_truth_list))):
        ground_truth = ground_truth_list.iloc[i]
        candidates = candidates_list.iloc[i]
        hit, mrr = 0.0, 0.0
        for j in range(len(candidates)):
            if ground_truth == candidates[j]:
                hit = 1.0
                mrr = 1.0 / (j + 1)
                break
        hits.append(hit)
        mrrs.append(mrr)
    return np.array(hits).mean(), np.array(mrrs).mean()

In [44]:
cal_hit_and_mrr(valid_UK_sessions_df['next_item'], sasrec_session_candidates_df['candidates'])

100%|██████████| 130665/130665 [00:05<00:00, 26002.54it/s]


(0.6061454865495733, 0.26247991444563873)

In [57]:
compare(1350)

***********last item**********
price                                                  8.99
brand                                                 Durex
locale                                                   UK
title     Durex 50 ml Play Perfect Glide Silicone Lubricant
Name: B01M07JRAV, dtype: object
***********next item**********
            price  brand locale  \
id                                
B078WVPQFX   9.99  Durex     DE   
B078WVPQFX   6.98  Durex     UK   
B078WVPQFX  12.50  Durex     ES   

                                                                                                               title  
id                                                                                                                    
B078WVPQFX  Durex Intense Orgasmic Gel - Stimulationsgel auf Wasserbasis für einen intensiveren Orgasmus - 1 x 10 ml  
B078WVPQFX                          Durex Intense Orgasmic Gel Lubricant Intensify for Her Warming Tightening Climax  
B078WVPQFX    

In [66]:
compare(2550)

***********prev items**********
price                                                                                                                10.0
brand                                                                                                            AproDerm
locale                                                                                                                 UK
title     Aproderm Colloidal Oat Cream - 500ml Pump - Paraffin Free Cream - Suitable for Dry Skin, Eczema and Psoriasis …
Name: B07QK1K57J, dtype: object
price                                                                                                                                      9.99
brand                                                                                                                                   D'Eyeko
locale                                                                                                                                       UK
title     D'Eyek

In [96]:
compare(3550)

***********prev items**********
price                                                                                                            10.99
brand                                                                                                         Highline
locale                                                                                                              UK
title     Highline Fidget Cube Fidget Toy for ADD and Stress Relief Fidget Sensory toys for Adults and Children (Camo)
Name: B06XDX6WX7, dtype: object
price                                                                                                             11.99
brand                                                                                                          Highline
locale                                                                                                               UK
title     Highline Fidget Cube Fidget Toy for ADD and Stress Relief Fidget Sensory toys for Adults a

In [69]:
compare(4550)

***********prev items**********
price                                                                               40000000.07
brand                                                                                 RoyalPack
locale                                                                                       UK
title     RoyalPack White Couch Roll 20", 2 PLY Hygiene Roll-40 Meters Long (Couch Roll 12Pack)
Name: B09HSMTVSV, dtype: object
price                                                                               40000000.07
brand                                                                                 RoyalPack
locale                                                                                       UK
title     RoyalPack White Couch Roll 20", 2 PLY Hygiene Roll-40 Meters Long (Couch Roll 4 Pack)
Name: B09HSNKYW2, dtype: object
price                                                                               40000000.07
brand                                   

In [70]:
compare(5550)

***********prev items**********
price                                                                                                         17.99
brand                                                                                                       QASIMOF
locale                                                                                                           UK
title     diasstro Lot of 30 50 100pc Random shoe charms for Clog Shoes Decorations Wristband Bracelet Party Favors
Name: B08XNC2ZJ1, dtype: object
price                                                                                                  7.57
brand                                                                                               QASIMOF
locale                                                                                                   UK
title     Lot of 30 50 100PCS Random Shoe Charms for Clog Shoes Decorations Wristband Bracelet Party Favors
Name: B08XNB33G4, dtype: object
price   

In [71]:
compare(6550)

***********prev items**********
price                                                                                                                                                                                         40000000.07
brand                                                                                                                                                                                            KASYDoFF
locale                                                                                                                                                                                                 UK
title     Clothes Dryer 1600W-1.7 Meters Portable Clothes Airer, 3-Tier Folding Electric Clothes Horse Rack Indoor, Energy Saving (Anion) Heated Clothes Airer Digital Automatic Timer for Apartment Home
Name: B083NFP5JS, dtype: object
price                                                                                                                           

In [72]:
compare(7550)

***********prev items**********
price                                                                                                                                                                                            6.5
brand                                                                                                                                                                                        L'Oreal
locale                                                                                                                                                                                            UK
title     L'Oreal Paris Men Expert One Twist Hair Colour, Dark Brown Hair Dye For Men. Mens Hair Dye For Men Dying Hair To Cover Grey Hair, Gives Quick Natural Looking Results - Shade 3 Dark Brown
Name: B09BGDBC76, dtype: object
            price                brand locale  \
id                                              
B07NBPSNC4   6.36                Combe     UK   
B

In [75]:
compare(9550)

***********prev items**********
            price  brand locale  \
id                                
B08PZCPRCT  15.50  MYLEE     DE   
B08PZCPRCT  13.00  MYLEE     UK   
B08PZCPRCT  12.99  MYLEE     ES   
B08PZCPRCT  14.99  MYLEE     FR   

                                                                                                                                                                                                              title  
id                                                                                                                                                                                                                   
B08PZCPRCT  Mylee 5-in-1 Builder stärkendes Gel - 15ml, UV LED Nagellack durchsichtige Schicht für starke Nagelspitzen, Form, strukturieren und Extensions aufbauen, Für Nagelkunst-Dekoration & Decals (Hell-Pink)  
B08PZCPRCT   Mylee 5 in 1 Builder Base Strengthening Gel 15ml, UV/LED Nail Polish Coat for Hard Strong Nails Tips &

In [86]:
compare_retromae(9550)

***********prev items**********
            price  brand locale  \
id                                
B08PZCPRCT  15.50  MYLEE     DE   
B08PZCPRCT  13.00  MYLEE     UK   
B08PZCPRCT  12.99  MYLEE     ES   
B08PZCPRCT  14.99  MYLEE     FR   

                                                                                                                                                                                                              title  
id                                                                                                                                                                                                                   
B08PZCPRCT  Mylee 5-in-1 Builder stärkendes Gel - 15ml, UV LED Nagellack durchsichtige Schicht für starke Nagelspitzen, Form, strukturieren und Extensions aufbauen, Für Nagelkunst-Dekoration & Decals (Hell-Pink)  
B08PZCPRCT   Mylee 5 in 1 Builder Base Strengthening Gel 15ml, UV/LED Nail Polish Coat for Hard Strong Nails Tips &

In [87]:
compare(11111)

***********prev items**********
            price    brand locale  \
id                                  
B072PRB6W1  29.98  Bestand     DE   
B072PRB6W1  24.98  Bestand     UK   

                                                                                                                                                                                                 title  
id                                                                                                                                                                                                      
B072PRB6W1                                                           Bestand 3 in 1 Aluminium Phone Stand, Ladestation, Dock Kompatibel mit iPhone, Apple Watch, Airpods and Other Smartphone - Silber  
B072PRB6W1  Bestand 3 in 1 for Apple iWatch Stand, Airpods Charger Dock, Phone Desktop Tablet Holder for Airpods, Apple Watch/iPhone X/8 Plus/8/7 Plus/iPad,Silver(Airpods Charging Case NOT Included)  
price          

In [88]:
compare_retromae(11111)

***********prev items**********
            price    brand locale  \
id                                  
B072PRB6W1  29.98  Bestand     DE   
B072PRB6W1  24.98  Bestand     UK   

                                                                                                                                                                                                 title  
id                                                                                                                                                                                                      
B072PRB6W1                                                           Bestand 3 in 1 Aluminium Phone Stand, Ladestation, Dock Kompatibel mit iPhone, Apple Watch, Airpods and Other Smartphone - Silber  
B072PRB6W1  Bestand 3 in 1 for Apple iWatch Stand, Airpods Charger Dock, Phone Desktop Tablet Holder for Airpods, Apple Watch/iPhone X/8 Plus/8/7 Plus/iPad,Silver(Airpods Charging Case NOT Included)  
price          

In [89]:
compare(12345)

***********prev items**********
                  price   brand locale  \
id                                       
B09T2TLHLJ  40000000.07  SHACOS     DE   
B09T2TLHLJ  40000000.07  SHACOS     UK   

                                                                                                                                                                                                     title  
id                                                                                                                                                                                                          
B09T2TLHLJ  SHACOS Teppich Läufer Flur rutschfest 50x150cm Küchenteppich Waschbar Teppichläufer Küche Schmutzläufer Flur Schmutzfangmatte Innenbereich für Küche, Eingang, Waschküche, Schlafzimmer (Grau)  
B09T2TLHLJ         SHACOS Runner Rug for Hallway Grey 50x150cm Non Slip Doormat Runner Machine Washable Heavy Duty Entrance Barrier Mat Absorbent Front Door Mat for Entryway Hallway Ki

In [90]:
compare_retromae(12345)

***********prev items**********
                  price   brand locale  \
id                                       
B09T2TLHLJ  40000000.07  SHACOS     DE   
B09T2TLHLJ  40000000.07  SHACOS     UK   

                                                                                                                                                                                                     title  
id                                                                                                                                                                                                          
B09T2TLHLJ  SHACOS Teppich Läufer Flur rutschfest 50x150cm Küchenteppich Waschbar Teppichläufer Küche Schmutzläufer Flur Schmutzfangmatte Innenbereich für Küche, Eingang, Waschküche, Schlafzimmer (Grau)  
B09T2TLHLJ         SHACOS Runner Rug for Hallway Grey 50x150cm Non Slip Doormat Runner Machine Washable Heavy Duty Entrance Barrier Mat Absorbent Front Door Mat for Entryway Hallway Ki

In [98]:
compare_retromae(12346)

***********prev items**********
price                                                                                                                                         137.0
brand                                                                                                                                Dell Computers
locale                                                                                                                                           UK
title     Complete Set of 22 in Monitors with Optiplex Quadcore Core i5 8GB Ram with 256 SSD GB Wifi Enabled Window 11 64 Bits Desktop PC (Renewed)
Name: B09V33NXBS, dtype: object
price                                                                                                                150.0
brand                                                                                                             Optiplex
locale                                                                                            

In [99]:
compare(12346) # 这个召回出来完全被关系

***********prev items**********
price                                                                                                                                         137.0
brand                                                                                                                                Dell Computers
locale                                                                                                                                           UK
title     Complete Set of 22 in Monitors with Optiplex Quadcore Core i5 8GB Ram with 256 SSD GB Wifi Enabled Window 11 64 Bits Desktop PC (Renewed)
Name: B09V33NXBS, dtype: object
price                                                                                                                150.0
brand                                                                                                             Optiplex
locale                                                                                            

In [100]:
compare(12347)

***********prev items**********
price                                                                                                                                                                                      23.89
brand                                                                                                                                                                                    fireowl
locale                                                                                                                                                                                        UK
title     fireowl Calming Dog Bed,Donut Cat Bed Anti Anxiety Dog Bed,Cozy Pet Dog Bed Plush Cuddler Soft Puppy Sofa,Cat Cushion Machine Washable Dog Sofa Bed (Ø60cm Outerdiameter, Beige Brown)
Name: B08H246SSB, dtype: object
price                                                                                                                                                               

In [101]:
compare_retromae(12347)

***********prev items**********
price                                                                                                                                                                                      23.89
brand                                                                                                                                                                                    fireowl
locale                                                                                                                                                                                        UK
title     fireowl Calming Dog Bed,Donut Cat Bed Anti Anxiety Dog Bed,Cozy Pet Dog Bed Plush Cuddler Soft Puppy Sofa,Cat Cushion Machine Washable Dog Sofa Bed (Ø60cm Outerdiameter, Beige Brown)
Name: B08H246SSB, dtype: object
price                                                                                                                                                               

In [102]:
compare_retromae(12348)

***********prev items**********
price                                                                                                                                                                       13.99
brand                                                                                                                                                                        CDJX
locale                                                                                                                                                                         UK
title     Jewelry Organizer,Metal T-Bar Necklace Display Stand Earrings Holder for Home, Hanging Pendant Rack with Marble Pattern Round Tray for Bracelets Rings Watches (A-Gold)
Name: B07XYXR2LR, dtype: object
              price                       brand locale  \
id                                                       
B082B4W9XW  2098.00  Amazonベーシック(Amazon Basics)     JP   
B082B4W9XW    18.13               Amazon Basics   

In [1]:
compare(12348)

NameError: name 'compare' is not defined

In [104]:
compare(12349)

***********prev items**********
            price         brand locale             title
id                                                      
1787633756   26.9  Bantam Press     DE  Jack Reacher, 27
1787633756   11.0  Bantam Press     UK  Jack Reacher, 27
            price         brand locale             title
id                                                      
1787633756   26.9  Bantam Press     DE  Jack Reacher, 27
1787633756   11.0  Bantam Press     UK  Jack Reacher, 27
***********next item**********
price                                                                                                                                                         8.89
brand                                                                                                                                                  Home Treats
locale                                                                                                                                                     

In [105]:
compare(12350)

***********prev items**********
price                                                    3.95
brand                                  Safety First Aid Group
locale                                                     UK
title     HypaBand Conforming Bandages - 5cm x 4M (Pack of 6)
Name: B0775P1ZSD, dtype: object
price                                                        4.53
brand                                      Safety First Aid Group
locale                                                         UK
title     HypaBand Conforming Bandages - 7.5 cm x 4 M (Pack of 6)
Name: B0775JDS56, dtype: object
price                                                        4.53
brand                                      Safety First Aid Group
locale                                                         UK
title     HypaBand Conforming Bandages - 7.5 cm x 4 M (Pack of 6)
Name: B0775JDS56, dtype: object
***********next item**********
price                                                            

In [106]:
compare(12351)

***********prev items**********
price                                                                                                                                                                                50.92
brand                                                                                                                                                                                 LEGO
locale                                                                                                                                                                                  UK
title     LEGO 71387 Super Mario Adventures with Luigi Starter Course Toy, Interactive Figure and Buildable Game Set &  71366 Super Mario Boomer Bill Barrage Expansion Set Buildable Game
Name: B09W1TFGFB, dtype: object
              price     brand locale  \
id                                     
B082WDQHZQ    42.99      LEGO     DE   
B082WDQHZQ  6027.00  レゴ(LEGO)     JP   
B082WDQHZQ    38.99      LEG

In [107]:
compare(12352)

***********prev items**********
price                                                                                                                                                                  7.99
brand                                                                                                                                                                COKEEY
locale                                                                                                                                                                   UK
title     Portable LED Camping lamp with Foldable Hook, Waterproof lamp, Battery Powered lamp, Portable Camping lamp, Applicable to Hurricane, Emergency, Family, shed, etc
Name: B0BJQ91MQF, dtype: object
price                                                                                                                                                                        19.99
brand                                                                

In [111]:
compare(12353)

***********prev items**********
locale                                                                                                                                                                                                                                                UK
title                                              Titanium Sports Premium Mouth Guard - Slim Fit Sports Mouth Guard with Case - Gum Shield for Jaw Protection in Contact Sports - Durable Boxing, MMA, Judo, Karate, Rugby, Hockey, Football Mouthguard
price                                                                                                                                                                                                                                               6.95
brand                                                                                                                                                                                                                        

In [112]:
compare(12354)

***********prev items**********
locale                                                                                UK
title       Tallon Festive Wonderland Flag-Shaped Christmas Tags, Gold Text - Pack of 20
price                                                                               3.99
brand                                                                             Tallon
color                                                                                NaN
size                                                                                 NaN
model                                                                                NaN
material                                                                             NaN
author                                                                               NaN
desc                                                                                 NaN
Name: B09JWVDZ59, dtype: object
locale                                        

In [113]:
compare(12355)

***********prev items**********
locale                                                                                UK
title                                  Stanley Safety Knife Blades (Pack of 10),2-11-987
price                                                                               2.61
brand                                                                            Stanley
color                                                                             Silver
size                                                                                   -
model                                                                           2-11-987
material                                                                             NaN
author                                                                               NaN
desc        Blades fit Springback knife and 99E original Stanley retractable blade knife
Name: B0044DAJ9Q, dtype: object
locale                                        

In [114]:
compare(12356)

***********prev items**********
           locale  \
id                  
B00AYULYLK     UK   
B00AYULYLK     ES   

                                                                                                                                                                         title  \
id                                                                                                                                                                               
B00AYULYLK  iDesign Shower Curtain with Magnetic Hem, Long Shower Curtain Made of Mould PEVA, Stylish and Water Repellent Shower Liner for Bathroom, Frost White, 183 x 183 cm   
B00AYULYLK                                                              iDesign Cortinas de ducha de tela, impermeables, resistentes al moho, 183.0 cm x 183.0 cm, Frost White   

            price        brand            color          size    model  \
id                                                                       
B00AYULYLK   6.00  Int

In [115]:
compare_retromae(12356)

***********prev items**********
           locale  \
id                  
B00AYULYLK     UK   
B00AYULYLK     ES   

                                                                                                                                                                         title  \
id                                                                                                                                                                               
B00AYULYLK  iDesign Shower Curtain with Magnetic Hem, Long Shower Curtain Made of Mould PEVA, Stylish and Water Repellent Shower Liner for Bathroom, Frost White, 183 x 183 cm   
B00AYULYLK                                                              iDesign Cortinas de ducha de tela, impermeables, resistentes al moho, 183.0 cm x 183.0 cm, Frost White   

            price        brand            color          size    model  \
id                                                                       
B00AYULYLK   6.00  Int

In [116]:
compare(12357)

***********prev items**********
locale                                                             UK
title                             Duzzit® Furniture Polish Wipes 50pk
price                                                             4.8
brand                                            151 Products Limited
color                                                          Orange
size                                                              NaN
model                                                         DZT005B
material                                                       Cotton
author                                                            NaN
desc        ABOUT THE PRODUCT: Each resealable pack contains 50 wipes
Name: B011UM5VQO, dtype: object
locale                                                                                                                                            UK
title       Mr Sheen Ultra Effective Wipes for Wood, Glass, Metal and Plastic, Spring M

In [117]:
compare(12358)

***********prev items**********
locale                                                                                                                                                                                                                                                                                                               UK
title                                                                                                                                                         Halloween Sweets, Trick or Treat Candy Mix - Trolli Dracula Gummy Teeth Bag 200g, Toxic Waste Smog Balls Bag 48g, Fruit Flavour Light-Up Pumpkin Lollipop
price                                                                                                                                                                                                                                                                                                              9.99
brand                           

In [118]:
compare(12359)

***********prev items**********
locale                  UK
title             The Scam
price                 0.99
brand       Boldwood Books
color                  NaN
size                   NaN
model                  NaN
material               NaN
author        Hunter, Evie
desc                   NaN
Name: B0B5CXGFGG, dtype: object
locale                  UK
title             The Scam
price                 0.99
brand       Boldwood Books
color                  NaN
size                   NaN
model                  NaN
material               NaN
author        Hunter, Evie
desc                   NaN
Name: B0B5CXGFGG, dtype: object
locale                  UK
title             The Scam
price                 0.99
brand       Boldwood Books
color                  NaN
size                   NaN
model                  NaN
material               NaN
author        Hunter, Evie
desc                   NaN
Name: B0B5CXGFGG, dtype: object
***********next item**********
locale                         

In [119]:
compare(12360)

***********prev items**********
locale                                   UK
title       DriPak White Vinegar, 500ml x 2
price                                  6.87
brand                               Dri Pak
color                                 White
size                     500 ml (Pack of 1)
model                                     .
material                                NaN
author                                  NaN
desc                                    NaN
Name: B0743JY18K, dtype: object
locale                                                                                                                                                            UK
title                                                                              500ml white vinegar, Spray bottle which can be used as window cleaning equipment.
price                                                                                                                                                            4.9
b

In [120]:
compare(12361)

***********prev items**********
           locale  \
id                  
B0B6FJKRVN     DE   
B0B6FJKRVN     UK   

                                                                          title  \
id                                                                                
B0B6FJKRVN  God Of War Ragnarök (PEGI 100% uncut Edition) (deutsche Verpackung)   
B0B6FJKRVN                                            God of War Ragnarök (PS4)   

            price        brand color size model material author  \
id                                                                
B0B6FJKRVN  79.99  Playstation   NaN  NaN   NaN      NaN    NaN   
B0B6FJKRVN  54.95  PlayStation   NaN  NaN   NaN      NaN    NaN   

                                                                                                                                                                                                     desc  
id                                                                           

In [121]:
√

***********prev items**********
locale                                                                                                                                                                                                    UK
title       YourZooki - Vitamin C Zooki - Vitamin C 1000mg Liquid Sachets - Alchohol Free, Vegan, Soy Free, Natural Ingredients - Immune System, Skin & Energy Support (Citrus Orange, 30 Count (Pack of 1))
price                                                                                                                                                                                                  39.99
brand                                                                                                                                                                                              yourzooki
color                                                                                                                                               

In [122]:
compare(12363)

***********prev items**********
locale                                                                                                                                                                                                                                                                                                                       UK
title                                                                                                                   INNOKIN Adept Zlide Vape Starter Kit, 3000mAh E-Cigarette Box Mod IP67 Waterproof, E-Cig 2ml Electronic Vapouriser Using Zenith Z-Coil, LED Display, MTL Smooth Vaping Auto-Wattage, No Nicotine, Black
price                                                                                                                                                                                                                                                                                                                     33.99
brand   

In [123]:
compare(12364)

***********prev items**********
           locale  \
id                  
B0B7W9LVCS     DE   
B0B7W9LVCS     UK   
B0B7W9LVCS     ES   
B0B7W9LVCS     IT   

                                                                                                                                                                                         title  \
id                                                                                                                                                                                               
B0B7W9LVCS                        MoKo Hülle für iPad 10. Generation 2022 mit iPencil Halter, Dünne iPad 10,9 Zoll Hülle Cover mit Auto Schlaf/Aufwach Funktion Schutzhülle iPad 2022, Schwarz   
B0B7W9LVCS  MoKo iPad 10th Generation Case with Pencil Holder iPad 10.9 Inch Case 2022, Soft TPU Smart Stand Back Cover Case for iPad 10th Generation, Support Touch ID&Auto Wake/Sleep, Black   
B0B7W9LVCS       MoKo Funda Compatible con Nuevo iPad 10ª Generac

In [124]:
compare(12365)

***********prev items**********
locale                                                                                                                                                                                 UK
title                        Skillmatics Guess in 10 Animal Planet | Card Game of Smart Questions | Super Fun for Travel & Family Game Night | Gifts, Stocking Stuffers for Ages 6 and Up
price                                                                                                                                                                               12.99
brand                                                                                                                                                                         Skillmatics
color                                                                                                                                                                                 NaN
size                                  

In [125]:
compare(12366)

***********prev items**********
           locale  \
id                  
B07L37F1RT     DE   
B07L37F1RT     UK   
B07L37F1RT     ES   

                                                                                        title  \
id                                                                                              
B07L37F1RT                         Revitale Advanced Salicylsäure- und Schwefel-Peeling-Seife   
B07L37F1RT                    Revitale Advanced Salicylic Acid & Sulphur Scrub Treatment Soap   
B07L37F1RT  Revitale Advanced Jabón para tratamiento exfoliante con ácido salicílico y azufre   

                  price     brand color                 size model material  \
id                                                                            
B07L37F1RT         7.99  REVITALE   NaN      80 g (1er Pack)   NaN      NaN   
B07L37F1RT         6.99  Revitale   NaN  1 Count (Pack of 1)   NaN      NaN   
B07L37F1RT  40000000.07  REVITALE   NaN  80 g (Paquete de 1)

In [126]:
compare(12367)

***********prev items**********
locale                                                                  UK
title       Sterling Silver Amethyst Purple Heart Crystal Pendant Necklace
price                                                                 1.92
brand                                                       Littleduckling
color                                                               Purple
size                                                               2.5*2cm
model                                                                 PN03
material                                                             Metal
author                                                                 NaN
desc                                                 Length (inches): 19.6
Name: B00JO7Q536, dtype: object
locale                                                                  UK
title       Sterling Silver Amethyst Purple Heart Crystal Pendant Necklace
price                               

In [127]:
compare_retromae(12367)

***********prev items**********
locale                                                                  UK
title       Sterling Silver Amethyst Purple Heart Crystal Pendant Necklace
price                                                                 1.92
brand                                                       Littleduckling
color                                                               Purple
size                                                               2.5*2cm
model                                                                 PN03
material                                                             Metal
author                                                                 NaN
desc                                                 Length (inches): 19.6
Name: B00JO7Q536, dtype: object
locale                                                                  UK
title       Sterling Silver Amethyst Purple Heart Crystal Pendant Necklace
price                               

In [128]:
compare(12368)

***********prev items**********
locale                                                                                                                                                                                                                                                                                                             UK
title                                                                                                                                        59" Halloween Giant Spider Decoration with 250 sqft Spider Web, Realistic Large 3D Scary Hairy Spider Prop Decor for Indoor Outdoor, Room, Yard, Garden, Halloween Party
price                                                                                                                                                                                                                                                                                                            8.99
brand                                 

In [129]:
compare(12369)

***********prev items**********
locale                                                                                                                                                                                                          UK
title                                                   Ear Plugs Soft Foam Ear Plugs for Sleeping Noise Cancelling Earplugs 5 Pairs Best Used for Sleeping, Travel, Concerts, Studying, Work, Loud Noise, Snoring
price                                                                                                                                                                                                         4.99
brand                                                                                                                                                                                                      CHONGNI
color                                                                                                                       